In [1]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


2023-11-28 12:56:28.149702: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to /Users/Mehrdadi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Mehrdadi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
data_file = open('/Users/Mehrdadi/Downloads/intents.json')
data = json.load(data_file)

Identifying Feature and Target for the NLP Model

In [6]:
words = []
classes = []
data_X = []
data_y = []

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_X.append(pattern)
        data_y.append(intent["tag"])
        
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes =sorted(set(classes))

In [7]:
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate (data_X):
    bow = []
    text = lemmatizer. lemmatize(doc. lower ())
    for word in words:
        bow. append(1) if word in text else bow. append (0)

    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded Boll and associated classes to training
    training. append ([bow, output_row])


random.shuffle(training)
training = np.array (training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np. array(list(training[:, 1]))

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len (train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])

print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 128)               7552      
                                                                 


 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_8 (Dense)             (None, 16)                1040      
                                                                 
Total params: 16,848
Trainable params: 16,848
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
2/2 [==============================] - 1s 3ms/step - loss: 2.8377 - accuracy: 0.0488
Epoch 2/150
2/2 [==============================] - 0s 4ms/step - loss: 2.5598 - accuracy: 0.1951
Epoch 3/150
2/2 [==============================] - 0s 4ms/step - loss: 2.3982 - accuracy: 0.2927
Epoch 4/150
2/2 [=======================

In [11]:
def clean_text (text):
    tokens = nltk. word_tokenize (text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

def bag_of_words (text, vocab):
    tokens = clean_text (text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate (vocab):
            if word == w:
                bow[idx] = 1
    return np.array (bow)


def pred_class (text, vocab, labels):
    bow = bag_of_words (text, vocab)
    result = model. predict (np.array ([bow]))[0] #extracting probabilities
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate (result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in dec
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]]) #Contains labels (tags) for highest probability 
    return return_list

def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I don't understand."
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random. choice(i["responses"])
                break
    return result

In [12]:
print("Press 0 if you don't want to chat with our ChatBot.")
while True:
    message = input("")
    if message == 0:
        break
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)

Press 0 if you don't want to chat with our ChatBot.
1/1 [==============================] - 0s 107ms/step
That is perfect!
1/1 [==============================] - 0s 29ms/step
So, everything's okay!
1/1 [==============================] - 0s 30ms/step
Hello
1/1 [==============================] - 0s 35ms/step
Sorry! I don't understand.
1/1 [==============================] - 0s 37ms/step
Hello
1/1 [==============================] - 0s 44ms/step
That is perfect!
1/1 [==============================] - 0s 31ms/step
I am feeling good, you?
1/1 [==============================] - 0s 35ms/step
That is perfect!
1/1 [==============================] - 0s 26ms/step
So, everything's okay!
1/1 [==============================] - 0s 28ms/step
So, everything's okay!
1/1 [==============================] - 0s 33ms/step
That is perfect!
1/1 [==============================] - 0s 28ms/step
So, everything's okay!
1/1 [==============================] - 0s 31ms/step
That is perfect!
1/1 [==========================